<a href="https://colab.research.google.com/github/soyuz13/testforgit/blob/master/KPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install mysqlclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime as dtime
from dataclasses import dataclass
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, func
from dateutil.relativedelta import relativedelta, SU, MO
from urllib.parse import quote_plus as urlquote
from sqlalchemy.orm import mapper, clear_mappers, sessionmaker
from enum import Enum
from kpi_config import DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_CHARSET, DB_NAME, DB_TYPE

In [ ]:
@dataclass
class Period:
  _today: dtime = dtime.datetime.now().date()
  
  @property
  def current_day(self):
    return self._today, self._today

  @property
  def current_week(self):
    min_date = self._today + relativedelta(weeks=0, weekday=MO(-1))
    max_date = self._today + relativedelta(weeks=0, weekday=SU(+1))
    return min_date, max_date

  @property
  def current_month(self):
    min_date = self._today + relativedelta(months=0, day=1)
    max_date = self._today + relativedelta(months=0, day=31)
    return min_date, max_date

  @property
  def current_year(self):
    min_date = self._today + relativedelta(years=0, day=1)
    max_date = self._today + relativedelta(years=0, day=366)
    return min_date, max_date

  @property
  def previous_day(self):
    return self._today + relativedelta(days=-1), self._today + relativedelta(days=-1)

  @property
  def previous_week(self):
    min_date = self._today + relativedelta(weeks=-1, weekday=MO(-1))
    max_date = self._today + relativedelta(weeks=-1, weekday=SU(+1))
    return min_date, max_date

  @property
  def previous_month(self):
    min_date = self._today + relativedelta(months=-1, day=1)
    max_date = self._today + relativedelta(months=-1, day=31)
    return min_date, max_date

  @property
  def previous_year(self):
    min_date = self._today + relativedelta(years=-1, day=1)
    max_date = self._today + relativedelta(years=-1, day=366)
    return min_date, max_date

  @property
  def previous_week_p(self):
    min_date = self._today + relativedelta(weeks=-1, weekday=MO(-1))
    max_date = self._today + relativedelta(weeks=-1)
    return min_date, max_date

  @property
  def previous_month_p(self):
    min_date = self._today + relativedelta(months=-1, day=1)
    max_date = self._today + relativedelta(months=-1)
    return min_date, max_date

  @property
  def previous_year_p(self):
    min_date = self._today + relativedelta(years=-1, day=1)
    max_date = self._today + relativedelta(years=-1)
    return min_date, max_date

  def any_week(self, date: dtime.datetime):
    min_date = date + relativedelta(weekday=MO(-1))
    max_date = date + relativedelta(weekday=SU(+1))
    return min_date, max_date

  def any_year(self, date: dtime.datetime):
    min_date = date + relativedelta(years=0, month=1, day=1)
    max_date = date + relativedelta(years=0, month=12, day=31)
    return min_date, max_date

  def any_month(self, date: dtime.datetime):
    min_date = date + relativedelta(months=0, day=1)
    max_date = date + relativedelta(months=0, day=31)
    return min_date, max_date


In [ ]:
class KPI:
  class R:
    pass

  class D:
    pass

  def __init__(self, data_connection):
    engine, self.session = data_connection
    # user, password, host, port = 'discount', 'Vl@zer52@7$', '95.154.87.128', '33306'
    # engine = create_engine(f'mysql://{user}:{urlquote(password)}@{host}:{port}/vlazer?charset=utf8', echo=True)
    # Session = sessionmaker(bind = engine)
    # self.session = Session()
    rawsales = Table('rawsales', MetaData(), Column("key", Integer, primary_key=True), autoload_with=engine)
    discountsales = Table('discountsales', MetaData(), Column("key", Integer, primary_key=True), autoload_with=engine)
    clear_mappers()
    mapper(self.R, rawsales, primary_key=[rawsales.c.key])
    mapper(self.D, discountsales, primary_key=[discountsales.c.key])
    pass

  @staticmethod
  def _check_result(query_result):
    query_result = query_result if query_result else 0
    return query_result

  def revenue(self, period: Period) -> float:
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1]
        )
    return round(self._check_result(res.first()[0]), 2)

  def checks(self, period: Period) -> float:
    res = self.session.query(
    func.count(func.distinct(self.R.sale_id))
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1]
        )
    return self._check_result(res.first()[0])

  def revenue_returnes(self, period: Period) -> float:
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.field0==3
    )
    return -round(self._check_result(res.first()[0]), 2)

  def cheks_returnes(self, period: Period) -> float:
    res = self.session.query(
    func.count(func.distinct(self.R.sale_id))
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.field0==3
    )
    return self._check_result(res.first()[0])

  def revenue_discount(self, period: Period) -> float:
    res = self.session.query(
    func.sum(self.D.fullsum)
    ).filter(
        self.D.date>=period[0], 
        self.D.date<=period[1]
    )
    return round(self._check_result(res.first()[0]), 2)

  def checks_discount(self, period: Period) -> float:
    res = self.session.query(
    func.count(func.distinct(self.D.sale_id))
    ).filter(
        self.D.date>=period[0], 
        self.D.date<=period[1]
        )
    return self._check_result(res.first()[0])

  def revenue_coffee(self, period: Period) -> float:
    goods = 'КР'
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def revenue_tea(self, period: Period) -> float:
    goods = 'ЧР'
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def revenue_presents(self, period: Period) -> float:
    goods = 'ПП '
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def quantity_presents(self, period: Period) -> float:
    goods = 'ПП '
    res = self.session.query(
    func.count(self.R.goods)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return self._check_result(res.first()[0])

  def revenue_bouquet(self, period: Period) -> float:
    goods = 'Б '
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def quantity_bouquet(self, period: Period) -> float:
    goods = 'Б '
    res = self.session.query(
    func.count(self.R.goods)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return self._check_result(res.first()[0])

  def revenue_basket(self, period: Period) -> float:
    goods = 'К '
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def quantity_basket(self, period: Period) -> float:
    goods = 'К '
    res = self.session.query(
    func.count(self.R.goods)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return self._check_result(res.first()[0])

  def revenue_packing(self, period: Period) -> float:
    goods = 'упаковка'
    res = self.session.query(
    func.sum(self.R.sum)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'%{goods}%')
        )
    return round(self._check_result(res.first()[0]), 2)

  def quantity_packing(self, period: Period) -> float:
    goods = 'упаковка'
    res = self.session.query(
    func.count(self.R.goods)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'%{goods}%')
        )
    return self._check_result(res.first()[0])

  def pockets_coffee(self, period: Period) -> float:
    goods = 'КР '
    res = self.session.query(
    func.count(self.R.sale_id)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return self._check_result(res.first()[0])

  def pockets_tea(self, period: Period) -> float:
    goods = 'ЧР '
    res = self.session.query(
    func.count(self.R.sale_id)
    ).filter(
        self.R.date>=period[0], 
        self.R.date<=period[1],
        self.R.goods.like(f'{goods}%')
        )
    return self._check_result(res.first()[0])

In [ ]:
class DataBaseType(Enum):
  mysql = 'mysql'
  sqlite = 'sqlite'

In [ ]:
class DataConn:
  def __init__(self, user: str='', password: str='', host: str='localhost', 
               port: int=3306, db_name: str='vlazer', charset: str='utf8',
               db_type: str='mysql'):
    self.user = user
    self.password = password
    self.host = host
    self.port = port
    self.db_name = db_name
    self.charset = charset
    self.db_type = db_type
    
  def __enter__(self):
    self.engine = create_engine(f'{self.db_type}://{self.user}:{urlquote(self.password)}@{self.host}:{self.port}/{self.db_name}?charset={self.charset}', echo=False)
    Session = sessionmaker(bind = self.engine)
    self.session = Session()
    return self.engine, self.session
    
  def __exit__(self, exc_type, exc_val, exc_tb):
    self.session.close()
    if exc_val:
      raise 

In [ ]:
with DataConn(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME) as data_connection:
  kpi = KPI(data_connection)
  period = Period()
  print(kpi.revenue(period.current_month), kpi.checks(period.current_month))

90762.82 126


# Новый раздел

In [ ]:
DB_TYPE

'mysql'